In [1]:
import os
import sys
import time
from datetime import date
from datetime import datetime
import time

import stoneburner
#//*** Custom Functions:
#//*** mr_clean_text(input_series)

# //*** Imports and Load Data
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#//*** Use the whole window in the IPYNB editor
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#//*** Maximize columns and rows displayed by pandas
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [2]:
input_filename  =".\\data\\wallstreetbets_comments.csv.zip"
output_filename = ".\\data\\processed_reddit_basic_v1.csv.zip"

#//*** Stock Ticker Symbols to track
symbols = ["CLOV","SOFI","WKHS","AMD","GME","X","AMC","CLNE","NIO","MU","SPCE","BB"]

#//*** Convert symbols to lower case
symbols = [x.lower() for x in symbols]
    
#//*** Load Clean and Prepare data for aggregation
start_time = time.time()
print("Reading Compressed CSV")
raw_df = pd.read_csv(input_filename,compression='zip' )
print(f"File Loaded: {round(time.time()-start_time,2)}s")

#//*** Convert UTC to date (not datetime)
#//** Second pass goes from 12-21 to 4-19
try:
    raw_df['created_utc'] = raw_df['created_utc'].apply(lambda x: date.fromtimestamp(x))
except:
    print()


raw_df['clean'] = stoneburner.remove_stop_words(stoneburner.tokenize_series(stoneburner.mr_clean_text(raw_df['body'],{"remove_empty":False})))

raw_df

Reading Compressed CSV
File Loaded: 2.31s
remove_empty False
Text Cleaning Time: 3.221360206604004
Tokenize Time: 35.08228659629822
Stop Words Time: 6.15438985824585


,score,total_awards_received,created_utc,is_submitter,author_fullname,body,id,link_id,parent_id,stickied,permalink,retrieved_on,subreddit,subreddit_id,hash,clean
0,1,0.0,2012-08-24,False,0,I will accept payments for my research...what'...,c5y9v5z,t3_yqtpn,t1_c5y49wz,False,0,1.429727e+09,wallstreetbets,t5_2th52,77c0f32dcf506571815f3d4839454f2b3e550f0e1efecd...,"[will, accept, payments, my, research, whats, ..."
1,3,0.0,2012-08-24,False,t2_4p1mf,"Because previously (until this post), when som...",c5yaaki,t3_yqtpn,t1_c5y9v5z,False,0,1.429728e+09,wallstreetbets,t5_2th52,b61949552c3a5d559111ba44d17a71113e408126dd198d...,"[previously, post, someone, claimed, looking, ..."
2,-2,0.0,2012-08-24,False,0,So you thought I was just going to give all of...,c5yahuo,t3_yqtpn,t1_c5yaaki,False,0,1.429728e+09,wallstreetbets,t5_2th52,97bf3c55f77337de3d6b83b05fc5601e2b346845b7b778...,"[thought, was, going, give, research, time, aw..."
3,2,0.0,2012-08-24,False,t2_3o5bc,I would also see some proof to back up your cl...,c5yaloj,t3_yqtpn,t1_c5y7jem,False,0,1.429728e+09,wallstreetbets,t5_2th52,5a3885658ca462a1fc0dd3b0af8858e183b05f4f474b84...,"[would, also, see, proof, back, claims, your, ..."
4,2,0.0,2012-08-24,False,t2_4p1mf,&gt; So you thought I was just going to give a...,c5yamfs,t3_yqtpn,t1_c5yahuo,False,0,1.429728e+09,wallstreetbets,t5_2th52,405314aad1e814100e822f7ac89274b17d66c4e11e5521...,"[because, post, chart, their, site, do, includ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399863,3,0.0,2021-06-25,False,t2_9z0opsh3,Nice,h32dw7m,t3_o7jx7p,t1_h329ycg,False,/r/wallstreetbets/comments/o7jx7p/fraternal_as...,1.624958e+09,wallstreetbets,t5_2th52,fdc96ffbf256523aec8846ae56321053c7ab751c99eb76...,[nice]
399864,3,0.0,2021-06-25,False,t2_qc6iq,Ah so horoscopes ARE real,h32dwab,t3_o7z71d,t3_o7z71d,False,/r/wallstreetbets/comments/o7z71d/amc_what_hap...,1.624958e+09,wallstreetbets,t5_2th52,20c5bee31dc23c8937aea64670a4157c6547621f763ff4...,"[ah, horoscopes, real]"
399865,3,0.0,2021-06-25,False,t2_bl7b6dkh,If one did this I would tell them Fuck Outta Here,h32dwip,t3_o7vagy,t1_h32daow,False,/r/wallstreetbets/comments/o7vagy/weekend_disc...,1.624958e+09,wallstreetbets,t5_2th52,96ac9390a4f9c8c36aeabf6d3f0358478ce7b3a295c9b6...,"[one, this, would, tell, fuck, outta]"
399866,4,0.0,2021-06-25,False,t2_5b0a37kn,Canada has still those stupid Covid restrictio...,h32dwhz,t3_o7vagy,t3_o7vagy,False,/r/wallstreetbets/comments/o7vagy/weekend_disc...,1.624958e+09,wallstreetbets,t5_2th52,6fa28767c5b75399225dd168a981c8570422fab812afab...,"[canada, still, stupid, covid, restrictions, p..."


In [3]:


def encode_comments(raw_df):
    import time
    
    print("Begin dataframe ticker symbol coding")
    start_time = time.time()
       
    
    
    #//*** Count each Stock mention add it to a dictionary of lists. Each list is filled with 0s. The Specific row index is updated with the relevant count. 
    #//*** This Generates a word count matrix
    stock_dict = {}

    #//*** Keep Track of Rows
    index = 0

    for row in raw_df.iterrows():

        #//*** Get the cleaned body text
        body = row[1]['clean']

        #//*** For Each Stock Symbol
        for stock in symbols:
            
            #//*** Check if Stock exists in Body
            if stock in body:

                #//*** Reset the stock counter
                count = 0

                #//*** Loop through body and county ticker mentions
                for word in body:
                    #//*** If word found increment count
                    if stock == word:
                        count += 1

                #//*** Check if symbol is in stock_dict
                if stock not in stock_dict.keys():    

                    #//*** If not, then build it
                    stock_dict[stock] = np.zeros(len(raw_df))

                #//*** Update the stock value at the 
                stock_dict[stock][index] = count

        #//*** Increment Index to keep with row index
        index +=1   

    #//*** Loop through the dictionary key and lists
    for col,values in stock_dict.items():

        #//*** Add each key (which is a stock ticker symbol) as a column using the list of ticker counts for Data
        raw_df[col] = values.astype('int')

    print(f"Encoding Time: {round(time.time()-start_time,2)}s")
    
    return raw_df


In [4]:
#//*** Encodes the dataframe with a count of Ticker symbols in each comment.
#//*** Called from update_subreddit(). This is broken out since we will likely need to adjust encoding parameters
def aggregate_comments(input_df):
    
    to_sum_cols = ['score','total_awards_received']
    to_count_col = ['author_fullname','link_id']
    
    
    df_cols = ['date','total_posts']
    
    rename_cols = {
        'total_awards_received' : 'awards',
        'author_fullname' : 'authors',
        'link_id' : 'threads'
    }
    
    #//*** Build the OUtput Dataframe Column names from the Columns to sum, the columns to count, and the stock ticker columns
    #//*** Loop through each list
    for cols in [ to_sum_cols, to_count_col, symbols ]:
        
        #//*** Get individual column name from each column list
        for col in cols:
            print
            #//*** Rename the column if in rename_col
            #//*** Add col to df_cols....The out_df column names
            if col in rename_cols.keys():
                df_cols.append(rename_cols[col])
            else:
                df_cols.append(col)
                
    print(df_cols)
    
    out_df = pd.DataFrame(columns = df_cols)
    
   
    #//*** Group 
    for group in input_df.groupby('created_utc'):
        
        loop_df = group[1].copy()
        
        loop_list = []
        
        #//*** Build the aggregated row for the Dataframe.
        #//*** 4 Parts: 
        #//******** 1.) Date & Total Posts
        #//******** 2.) Columns to sum
        #//******** 3.) Columns to count
        #//******** 4.) Stock Ticker columns to sum
        
        #//********************************************
        #//******** 1.) Date & Total Posts
        #//********************************************
        #//*** Add the Date
        loop_list.append(group[0])
        
        #//*** Add Total number of posts
        loop_list.append(len(loop_df))
        
        #//********************************************
        #//******** 2.) Columns to sum
        #//********************************************
        for col in to_sum_cols:
            loop_list.append(loop_df[col].sum())

            
        #//********************************************
        #//******** 3.) Columns to count
        #//********************************************
        for col in to_count_col:
            loop_list.append(len(loop_df[col].unique()))
    
        
        #//********************************************
        #//******** 4.) Stock Ticker columns to sum
        #//********************************************
        for col in symbols:
            loop_list.append(loop_df[col].sum())

        #print(len(out_df.columns),len(loop_list))
        #print(out_df.columns)
        out_df.loc[len(out_df.index)] = loop_list 

    return out_df



#for col in df.columns[16:]:
#    print(df[df[col] > 0 ].iloc[0]['created_utc'],col)

In [5]:
#//*** Encode Comments
df = encode_comments(raw_df)

#//*** Aggregate and Process Comments
ag_df = aggregate_comments(df)
ag_df

Begin dataframe ticker symbol coding
Encoding Time: 19.41s
['date', 'total_posts', 'score', 'awards', 'authors', 'threads', 'clov', 'sofi', 'wkhs', 'amd', 'gme', 'x', 'amc', 'clne', 'nio', 'mu', 'spce', 'bb']


,date,total_posts,score,awards,authors,threads,clov,sofi,wkhs,amd,gme,x,amc,clne,nio,mu,spce,bb
0,2012-08-24,29,80,0.0,13,6,0,0,0,0,0,0,0,0,0,0,0,0
1,2012-08-25,19,18,0.0,7,3,0,0,0,0,0,0,0,0,0,0,0,0
2,2012-08-26,13,11,0.0,9,5,0,0,0,0,0,0,0,0,0,0,0,0
3,2012-08-27,11,21,0.0,6,6,0,0,0,0,0,0,0,0,0,0,0,0
4,2012-08-28,5,9,0.0,4,3,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,2021-06-21,567,2878,8.0,429,89,10,0,7,4,9,1,10,7,5,3,0,12
447,2021-06-22,557,3076,2.0,400,113,14,2,2,5,8,1,10,7,1,0,0,10
448,2021-06-23,694,4543,7.0,488,118,21,0,11,3,12,0,9,3,6,0,3,8
449,2021-06-24,513,2857,5.0,397,72,18,0,12,4,1,0,6,3,0,1,1,5


In [6]:
#//*** Write File to disk

ag_df.to_csv(output_filename,compression="zip",index=False) 